In [3]:
!python --version

Python 2.7.11


In [9]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint

# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.split(' ')]
    return LabeledPoint(values[0], values[1:])

data = sc.textFile("/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb3f-2e8796b7817263-c965b24fe08f/notebook/work/sample_svm_data.txt")
#print data.collect()
parsedData = data.map(parsePoint)

# Build the model
model = SVMWithSGD.train(parsedData, iterations=100)

# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

# Save and load model
#model.save(sc, "target/tmp/pythonSVMWithSGDModel")
#sameModel = SVMModel.load(sc, "target/tmp/pythonSVMWithSGDModel")

Training Error = 0.38198757764


In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint

# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.split(' ')]
    return LabeledPoint(values[0], values[1:])

data = sc.textFile("/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb3f-2e8796b7817263-c965b24fe08f/notebook/work/sample_svm_data.txt")

parsedData = data.map(parsePoint)

# Build the model
model = LogisticRegressionWithLBFGS.train(parsedData)

# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda lp: lp[0] != lp[1]).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

In [6]:
!rm -r sample_svm_data.txt 

rm: cannot remove ‘sample_svm_data.txt’: No such file or directory


In [11]:
! wget https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_libsvm_data.txt

--2017-08-05 13:43:21--  https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_libsvm_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.180.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.180.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104736 (102K) [text/plain]
Saving to: ‘sample_libsvm_data.txt.1’

100%[======================================>] 104,736     --.-K/s   in 0.02s   

2017-08-05 13:43:22 (5.91 MB/s) - ‘sample_libsvm_data.txt.1’ saved [104736/104736]



In [ ]:
### NAvie Bayes

In [13]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.util import MLUtils



# Load and parse the data file.
data = MLUtils.loadLibSVMFile(sc, "/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb3f-2e8796b7817263-c965b24fe08f/notebook/work/sample_libsvm_data.txt.1")

# Split data approximately into training (60%) and test (40%)
training, test = data.randomSplit([0.6, 0.4])

# Train a naive Bayes model.
model = NaiveBayes.train(training, 1.0)

# Make prediction and test accuracy.
predictionAndLabel = test.map(lambda p: (model.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda pl: pl[0] == pl[1]).count() / test.count()
print('model accuracy {}'.format(accuracy))


model accuracy 0.975609756098


In [ ]:
##Gradient Boost Tree Algorithms

In [15]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file.
data = MLUtils.loadLibSVMFile(sc, "/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb3f-2e8796b7817263-c965b24fe08f/notebook/work/sample_libsvm_data.txt.1")
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a GradientBoostedTrees model.
#  Notes: (a) Empty categoricalFeaturesInfo indicates all features are continuous.
#         (b) Use more iterations in practice.
model = GradientBoostedTrees.trainClassifier(trainingData,
                                             categoricalFeaturesInfo={}, numIterations=3)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification GBT model:')
print(model.toDebugString())

Test Error = 0.037037037037
Learned classification GBT model:
TreeEnsembleModel classifier with 3 trees

  Tree 0:
    If (feature 406 <= 72.0)
     If (feature 100 <= 165.0)
      Predict: -1.0
     Else (feature 100 > 165.0)
      Predict: 1.0
    Else (feature 406 > 72.0)
     Predict: 1.0
  Tree 1:
    If (feature 433 <= 0.0)
     If (feature 379 <= 251.0)
      If (feature 263 <= 145.0)
       Predict: -0.4768116880884702
      Else (feature 263 > 145.0)
       Predict: -0.4768116880884703
     Else (feature 379 > 251.0)
      Predict: 0.4768116880884694
    Else (feature 433 > 0.0)
     Predict: 0.4768116880884701
  Tree 2:
    If (feature 434 <= 0.0)
     If (feature 627 <= 0.0)
      Predict: 0.4381935810427206
     Else (feature 627 > 0.0)
      If (feature 209 <= 0.0)
       Predict: -0.4381935810427206
      Else (feature 209 > 0.0)
       Predict: -0.4381935810427206
    Else (feature 434 > 0.0)
     If (feature 188 <= 252.0)
      Predict: 0.4381935810427207
     Else (fea

In [ ]:
##Decision Tree

In [16]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, "/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb3f-2e8796b7817263-c965b24fe08f/notebook/work/sample_libsvm_data.txt.1")
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification tree model:')
print(model.toDebugString())

Test Error = 0.0322580645161
Learned classification tree model:
DecisionTreeModel classifier of depth 2 with 5 nodes
  If (feature 406 <= 20.0)
   If (feature 100 <= 165.0)
    Predict: 0.0
   Else (feature 100 > 165.0)
    Predict: 1.0
  Else (feature 406 > 20.0)
   Predict: 1.0



In [17]:
##Random Forest

In [18]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, "/gpfs/global_fs01/sym_shared/YPProdSpark/user/sb3f-2e8796b7817263-c965b24fe08f/notebook/work/sample_libsvm_data.txt.1")
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
#  Note: Use larger numTrees in practice.
#  Setting featureSubsetStrategy="auto" lets the algorithm choose.
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification forest model:')
print(model.toDebugString())

Test Error = 0.0333333333333
Learned classification forest model:
TreeEnsembleModel classifier with 3 trees

  Tree 0:
    If (feature 512 <= 0.0)
     If (feature 315 <= 144.0)
      If (feature 486 <= 192.0)
       Predict: 1.0
      Else (feature 486 > 192.0)
       Predict: 0.0
     Else (feature 315 > 144.0)
      Predict: 0.0
    Else (feature 512 > 0.0)
     Predict: 0.0
  Tree 1:
    If (feature 351 <= 2.0)
     Predict: 0.0
    Else (feature 351 > 2.0)
     Predict: 1.0
  Tree 2:
    If (feature 301 <= 0.0)
     If (feature 291 <= 102.0)
      Predict: 1.0
     Else (feature 291 > 102.0)
      Predict: 0.0
    Else (feature 301 > 0.0)
     Predict: 0.0

